<a href="https://colab.research.google.com/github/Likelipop/DQL_Trading_based/blob/main/new_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vnstock3

In [ ]:
!pip install stable-baselines3 gymnasium

In [2]:
import pandas as pd
from vnstock import Vnstock


    Khi tiếp tục sử dụng Vnstock3, bạn xác nhận rằng bạn đã đọc, hiểu và đồng ý với Chính sách quyền riêng tư và Điều khoản, điều kiện về giấy phép sử dụng Vnstock3.

    Chi tiết:

    - Giấy phép sử dụng phần mềm: https://vnstocks.com/docs/tai-lieu/giay-phep-su-dung
    - Chính sách quyền riêng tư: https://vnstocks.com/docs/tai-lieu/chinh-sach-quyen-rieng-tu
    


**PART 1: DATA PREPARATION**

Ở đây, ta sẽ xây dựng một lớp để xử lý việc tải và xử lý dữ liệu từ VNSTOCK3

In [7]:
def load_data(symbol, date_start, date_end):
    """Loads data from vnstock3 for a given symbol.

    Args:
      symbol: The stock symbol (e.g., 'TCB', 'VIC').
      date_start: starting query date: 'YYYY-MM-DD'.
      date_end: ending query date: 'YYYY-MM-DD'.

    Returns:
      A pandas DataFrame containing the historical data for the given symbol,
      or None if an error occurs.
    """
    try:
        # Replace with actual vnstock3 data loading logic
        # Example using a placeholder CSV file
        # df = pd.read_csv(f"path/to/data/{symbol}.csv")
        stock = Vnstock().stock(symbol=symbol, source="VCI")
        return stock.quote.history(start=date_start, end=date_end)
    except FileNotFoundError:
        print(f"Error: Data for symbol '{symbol}' not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [8]:
df_data = load_data(symbol = "VNM",date_start = "2015-01-01", date_end = "2025-01-01")
df_data

2025-02-04 16:24:16 - vnstock.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,time,open,high,low,close,volume
0,2015-01-05,30.34,30.50,30.18,30.34,70290
1,2015-01-06,30.34,30.66,30.03,30.34,100890
2,2015-01-07,30.34,31.13,30.34,30.97,194130
3,2015-01-08,30.97,31.29,30.82,31.13,70650
4,2015-01-09,31.45,32.55,31.29,32.55,290390
...,...,...,...,...,...,...
2495,2024-12-26,64.20,64.20,63.80,63.80,1711855
2496,2024-12-27,63.90,64.10,63.70,63.80,2445187
2497,2024-12-30,63.60,63.70,63.30,63.30,1492983
2498,2024-12-31,63.40,63.80,63.30,63.40,1643941


In [24]:
close_column = df_data["close"].to_numpy()
Xtrain, Xtest = close_column[:int(len(close_column)*0.8)], close_column[int(len(close_column)*0.8):]
print(f"X_train: {Xtrain.shape}, X_test: {Xtest.shape}")

X_train: (2000,), X_test: (500,)


`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


**PART 2: ENVIROMENT**

In [44]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import matplotlib.pyplot as plt
class TradingEnv(gym.Env):
    def __init__(self, window_size):
        super(TradingEnv, self).__init__()
        self.data = Xtrain
        self.window_size = window_size
        self.current_step = self.window_size
        self.action_space = spaces.Discrete(3)  # [0: Sell, 1: Hold, 2: Buy]
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(self.window_size,), dtype=np.float32
        )
        self.inventory = []
        self.done = False
        self.logs = {'states_buy': [], 'states_sell': []}
        self.total_profit = 0

    def reset(self):
        self.current_step = 0
        self.logs = {'states_buy': [], 'states_sell': []}
        self.inventory = []
        self.done = False
        self.total_profit = 0

        return self._next_observation(),{}

    def _next_observation(self):
        if self.current_step < self.window_size:
            delta = self.window_size - self.current_step
            obs = np.concatenate(([0]*delta, self.data[:self.current_step]))
        else:
            obs = self.data[self.current_step - self.window_size:self.current_step]

        normalized_obs = self._normalize(obs)
        return normalized_obs

    def _normalize(self, data):
        return 1 / (1 + np.exp(-data))  # Sigmoid normalization

    def step(self, action):
        reward = 0
        prev_price = self.data[self.current_step - 1]
        current_price = self.data[self.current_step]

        # Action logic
        if (action == 0) and (len(self.inventory) > 0):  # Sell
            bought_price = self.inventory.pop(0)
            self.logs['states_sell'].append(self.current_step)
            reward = max(current_price - bought_price, 0)

        elif action == 2:  # Buy
            self.inventory.append(current_price)
            reward = 0
        # Hold action doesn't change inventory or immediate reward

        self.current_step += 1

        if self.current_step >= len(self.data):
            self.done = True

        observation = self._next_observation()
        info = {}

        if self.done:
          self.render()

        return observation, reward, self.done, info

    def render(self, mode='rgb'):
        plt.figure(figsize=(12, 6))
        plt.plot(self.data, color='r', lw=2.)
        plt.plot(self.data, 'v', markersize=10, color='m', label='Selling signal', markevery=self.logs["states_sell"])
        plt.plot(self.data, "^", markersize=10, color="k", label="Buy signal", markevery = self.logs["states_buy"])
        plt.title('Total gains: %f'%self.total_profit)
        plt.legend()
        plt.show()



`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [28]:
from gymnasium.envs.registration import register
register(
    id='TradingBot-v2',
    entry_point=lambda: TradingEnv,  # Using lambda for parameters
)

In [51]:
import os
import random
import numpy as np
from stable_baselines3 import DQN

# Initialize environment
env = TradingEnv(window_size=10)
# File names
model_filename = 'dqn_trading_model.zip'
replay_buffer_filename = 'dqn_replay_buffer.pkl'

# Load or create a model
if os.path.exists(model_filename):
    model = DQN.load(model_filename, env=env)
    print("Model loaded.")
    if os.path.exists(replay_buffer_filename):
        model.load_replay_buffer(replay_buffer_filename)
        print("Replay buffer loaded.")
    else:
        print("No replay buffer found. Starting with an empty buffer.")
else:
    model = DQN('MlpPolicy', env, verbose=1)
    print("New model created.")

# Set episodes per session
episode_count = 1

for episode in range(episode_count):
    obs,_ = env.reset()
    done = False

    while not done:
        # Get action from the model
        action, _ = model.predict(obs, deterministic=False)

        action = np.array([action])

        # Take the action in the environment
        next_obs, reward, done, info = env.step(action[0])

        # Add the transition to the replay buffer
        model.replay_buffer.add(obs, next_obs, action, reward, done, [info])

        obs = next_obs

        # Train the model if there are enough samples
        if model.replay_buffer.size() > model.batch_size:
            model.train(batch_size=model.batch_size, gradient_steps=1)

    model.save(model_filename)
    model.save_replay_buffer(replay_buffer_filename)
    print(f"Episode {episode + 1} completed. Model and replay buffer saved.")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
New model created.


`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


AttributeError: 'DQN' object has no attribute '_logger'

In [52]:
from stable_baselines3.common.callbacks import BaseCallback

class SaveOnEpisodeEndCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(SaveOnEpisodeEndCallback, self).__init__(verbose)

    def _on_step(self):
        # Check if the episode is done
        if self.locals.get('dones', [False])[0]:
            # Save the model and replay buffer
            self.model.save(model_filename)
            self.model.save_replay_buffer(replay_buffer_filename)
            if self.verbose > 0:
                print(f"Episode {self.num_timesteps} completed. Model and replay buffer saved.")
        return True


In [53]:
import os
import random
import numpy as np
import torch
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback

# Set random seeds


# Initialize the environment
env = TradingEnv(window_size=10)

# File names
model_filename = 'dqn_trading_model.zip'
replay_buffer_filename = 'dqn_replay_buffer.pkl'

# Load or create a model
if os.path.exists(model_filename):
    model = DQN.load(model_filename, env=env, verbose=1)
    print("Model loaded.")
    if os.path.exists(replay_buffer_filename):
        model.load_replay_buffer(replay_buffer_filename)
        print("Replay buffer loaded.")
    else:
        print("No replay buffer found. Starting with an empty buffer.")
else:
    model = DQN('MlpPolicy', env, verbose=1)
    print("New model created.")

# Custom callback to save the model at the end of each episode
class SaveOnEpisodeEndCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(SaveOnEpisodeEndCallback, self).__init__(verbose)

    def _on_step(self):
        # Check if the episode is done
        if self.locals.get('dones', [False])[0]:
            # Save the model and replay buffer
            self.model.save(model_filename)
            self.model.save_replay_buffer(replay_buffer_filename)
            if self.verbose > 0:
                print(f"Episode at timestep {self.num_timesteps} completed. Model and replay buffer saved.")
        return True

callback = SaveOnEpisodeEndCallback(verbose=1)

# Train the model for one episode
total_timesteps_per_episode = len(env.data) - env.window_size  # Steps in one episode

model.learn(
    total_timesteps=total_timesteps_per_episode,
    callback=callback,
    reset_num_timesteps=False
)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
New model created.


TypeError: TradingEnv.reset() got an unexpected keyword argument 'seed'